In [131]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
import requests

In [132]:
load_dotenv()
ALL_CONTACTS_VIEW_ID = os.getenv('ALL_CONTACTS_VIEW_ID')
ALL_ACCOUNTS_VIEW_ID = os.getenv('ALL_ACCOUNTS_VIEW_ID')
ALL_DEALS_VIEW_ID = os.getenv('ALL_DEALS_VIEW_ID')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')

In [133]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [134]:
def extract_data(entity, view_id, output_dir):
    FRESHSALES_API_KEY = os.getenv('FRESHSALES_API_KEY')
    SALES_BUNDLE_ALIAS = os.getenv('SALES_BUNDLE_ALIAS')

    if not FRESHSALES_API_KEY or not SALES_BUNDLE_ALIAS:
        raise ValueError("FRESHSALES_API_KEY or SALES_BUNDLE_ALIAS environment variable is missing.")

    page = 1
    all_data = []
    while True:
        url = f"https://{SALES_BUNDLE_ALIAS}/api/{entity}/view/{view_id}?page={page}"
        headers = {
            "Authorization": f"Token token={FRESHSALES_API_KEY}",
            "Content-Type": "application/json"
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data[entity])

            # Check if there are more pages
            if page >= data['meta']['total_pages']:
                break
            page += 1
        else:
            print(f"Failed to extract data for {entity}. Status Code: {response.status_code}")
            break

    output_file = f"{output_dir}/{entity}.json"
    with open(output_file, "w") as file:
        json.dump(all_data, file)
    print(f"Successfully extracted data for {entity} to {output_file}")

extract_data("contacts", ALL_CONTACTS_VIEW_ID, OUTPUT_DIR)
extract_data("sales_accounts", ALL_ACCOUNTS_VIEW_ID, OUTPUT_DIR)
extract_data("deals", ALL_DEALS_VIEW_ID, OUTPUT_DIR)

Successfully extracted data for contacts to ./data/contacts.json
Successfully extracted data for sales_accounts to ./data/sales_accounts.json
Successfully extracted data for deals to ./data/deals.json


In [147]:
def load_data(data_folder):
    contacts_data_path = os.path.join(data_folder, 'contacts.json')
    deals_data_path = os.path.join(data_folder, 'deals.json')
    accounts_data_path = os.path.join(data_folder, 'sales_accounts.json')

    with open(contacts_data_path) as contacts_file:
        contacts_data = json.load(contacts_file)
    with open(deals_data_path) as deals_file:
        deals_data = json.load(deals_file)
    with open(accounts_data_path) as accounts_file:
        accounts_data = json.load(accounts_file)

    return contacts_data, deals_data, accounts_data

In [148]:
def transform_data(contacts_data, deals_data, accounts_data):
    # Convert each JSON list to a DataFrame
    contacts_df = pd.DataFrame(contacts_data)
    deals_df = pd.DataFrame(deals_data)
    accounts_df = pd.DataFrame(accounts_data)
    return contacts_df, deals_df, accounts_df


In [149]:
contacts_data, deals_data, accounts_data = load_data(OUTPUT_DIR)
contacts_df, deals_df, accounts_df = transform_data(contacts_data, deals_data, accounts_data)

In [150]:
# Find empty columns in contacts_df
empty_columns_contacts = contacts_df.columns[contacts_df.isna().all()].tolist()
non_empty_columns_contacts = contacts_df.columns[~contacts_df.isna().all()].tolist()

# Print summary for contacts_df
print("Total number of columns in contacts_df:", contacts_df.shape[1])
print("Number of empty columns in contacts_df:", len(empty_columns_contacts))
print("Number of non-empty columns in contacts_df:", len(non_empty_columns_contacts))
print("List of non-empty columns in contacts_df:", non_empty_columns_contacts)

Total number of columns in contacts_df: 71
Number of empty columns in contacts_df: 35
Number of non-empty columns in contacts_df: 36
List of non-empty columns in contacts_df: ['id', 'first_name', 'last_name', 'display_name', 'avatar', 'job_title', 'city', 'state', 'country', 'email', 'emails', 'time_zone', 'mobile_number', 'lead_score', 'open_deals_amount', 'won_deals_amount', 'links', 'custom_field', 'created_at', 'updated_at', 'won_deals_count', 'open_deals_count', 'last_assigned_at', 'linkedin', 'is_deleted', 'team_user_ids', 'subscription_status', 'subscription_types', 'customer_fit', 'record_type_id', 'whatsapp_subscription_status', 'sms_subscription_status', 'system_tags', 'mcr_id', 'phone_numbers', 'tags']


In [151]:
# Find empty columns in accounts_df
empty_columns_accounts = accounts_df.columns[accounts_df.isna().all()].tolist()
non_empty_columns_accounts = accounts_df.columns[~accounts_df.isna().all()].tolist()

# Print summary for accounts_df
print("Total number of columns in accounts_df:", accounts_df.shape[1])
print("Number of empty columns in accounts_df:", len(empty_columns_accounts))
print("Number of non-empty columns in accounts_df:", len(non_empty_columns_accounts))
print("List of non-empty columns in accounts_df:", non_empty_columns_accounts)

Total number of columns in accounts_df: 44
Number of empty columns in accounts_df: 14
Number of non-empty columns in accounts_df: 30
List of non-empty columns in accounts_df: ['id', 'name', 'address', 'city', 'state', 'zipcode', 'country', 'number_of_employees', 'annual_revenue', 'website', 'owner_id', 'phone', 'open_deals_amount', 'open_deals_count', 'won_deals_amount', 'won_deals_count', 'facebook', 'twitter', 'linkedin', 'links', 'custom_field', 'created_at', 'updated_at', 'avatar', 'last_assigned_at', 'is_deleted', 'team_user_ids', 'record_type_id', 'domains', 'tags']


In [143]:
# Find empty columns in deals_df
empty_columns_deals = deals_df.columns[deals_df.isna().all()].tolist()
non_empty_columns_deals = deals_df.columns[~deals_df.isna().all()].tolist()

# Print summary for deals_df
print("Total number of columns in deals_df:", deals_df.shape[1])
print("Number of empty columns in deals_df:", len(empty_columns_deals))
print("Number of non-empty columns in deals_df:", len(non_empty_columns_deals))
print("List of non-empty columns in deals_df:", non_empty_columns_deals)

Total number of columns in deals_df: 40
Number of empty columns in deals_df: 11
Number of non-empty columns in deals_df: 29
List of non-empty columns in deals_df: ['id', 'name', 'amount', 'base_currency_amount', 'expected_close', 'closed_date', 'stage_updated_time', 'custom_field', 'probability', 'updated_at', 'created_at', 'deal_pipeline_id', 'deal_stage_id', 'age', 'links', 'collaboration', 'last_assigned_at', 'expected_deal_value', 'is_deleted', 'team_user_ids', 'fc_widget_collaboration', 'forecast_category', 'deal_prediction', 'deal_prediction_last_updated_at', 'record_type_id', 'has_products', 'products', 'deal_price_adjustments', 'tags']


In [144]:
deals_df

,id,name,amount,base_currency_amount,expected_close,closed_date,stage_updated_time,custom_field,probability,updated_at,...,deal_prediction,deal_prediction_last_updated_at,record_type_id,freddy_forecast_metrics,last_deal_prediction,has_products,products,deal_price_adjustments,rotten_days,tags
0,202000322318,Multi-channeled multi-state throughput Deal,99325.0,99325.0,2024-04-17,None,2024-11-19T01:49:38+01:00,"{'cf_deal_size': 100000, 'cf_probability_of_cl...",11,2024-11-19T10:11:21+01:00,...,0,None,203002646931,None,None,False,[],[],None,[]
1,202000322321,Multi-layered real-time adapter Deal,89509.0,89509.0,2025-08-26,None,2024-11-19T01:49:39+01:00,"{'cf_deal_size': 90000, 'cf_probability_of_clo...",74,2024-11-19T10:15:41+01:00,...,0,None,203002646931,None,None,False,[],[],None,[]
2,202000322319,Reactive dynamic secured line Deal,80707.0,80707.0,2024-02-09,None,2024-11-19T01:49:38+01:00,"{'cf_deal_size': 90000, 'cf_probability_of_clo...",59,2024-11-19T10:09:54+01:00,...,0,None,203002646931,None,None,False,[],[],None,[]
3,202000322326,Phased neutral frame Deal,80553.0,80553.0,2025-10-21,None,2024-11-19T01:49:40+01:00,"{'cf_deal_size': 90000, 'cf_probability_of_clo...",35,2024-11-19T10:14:37+01:00,...,0,None,203002646931,None,None,False,[],[],None,[]
4,202000322323,Multi-channeled 24hour architecture Deal,76439.0,76439.0,2025-07-15,2024-11-19,2024-11-19T01:49:39+01:00,"{'cf_deal_size': 80000, 'cf_probability_of_clo...",23,2024-11-19T10:12:46+01:00,...,7,2024-11-19T01:49:39+01:00,203002646931,None,None,False,[],[],None,[]
5,202000322315,Virtual 3rdgeneration application Deal,69590.0,69590.0,2024-12-06,None,2024-11-19T01:49:37+01:00,"{'cf_deal_size': 70000, 'cf_probability_of_clo...",13,2024-11-19T10:16:59+01:00,...,0,None,203002646931,None,None,False,[],[],None,[]
6,202000322322,Networked neutral emulation Deal,54250.0,54250.0,2025-10-07,2025-10-31,2024-11-19T01:49:39+01:00,"{'cf_deal_size': 60000, 'cf_probability_of_clo...",27,2024-11-19T10:12:22+01:00,...,6,2024-11-19T01:49:39+01:00,203002646931,None,None,False,[],[],None,[]
7,202000322325,Progressive executive circuit Deal,43615.0,43615.0,2025-03-06,None,2024-11-19T01:49:40+01:00,"{'cf_deal_size': 50000, 'cf_probability_of_clo...",89,2024-11-19T10:06:42+01:00,...,0,None,203002646931,None,None,False,[],[],None,[]
8,202000322320,Multi-lateral empowering customer loyalty Deal,41360.0,41360.0,2025-10-17,None,2024-11-19T01:49:39+01:00,"{'cf_deal_size': 500000, 'cf_probability_of_cl...",77,2024-11-19T10:11:59+01:00,...,0,None,203002646931,None,None,False,[],[],None,[]
9,202000322316,Adaptive human-resource paradigm Deal,39558.0,39558.0,2025-09-26,None,2024-11-19T01:49:38+01:00,"{'cf_deal_size': 40000, 'cf_probability_of_clo...",20,2024-11-19T10:16:04+01:00,...,0,None,203002646931,None,None,False,[],[],None,[]
